# Financial Analytics & Revenue Forecasting

## Comprehensive Analysis using Python

This notebook performs:
- Financial ratio analysis
- Revenue forecasting with multiple models
- Trend analysis and seasonality detection
- Comprehensive visualization and insights
- Report generation

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Import custom modules
import sys
sys.path.insert(0, '../src')

from financial_ratios import (
    calculate_current_ratio, calculate_quick_ratio, calculate_profit_margin,
    calculate_roa, calculate_roe, calculate_debt_to_equity, calculate_asset_turnover
)
from revenue_forecast import (
    MovingAverageForecaster, ExponentialSmoothingForecaster,
    LinearRegressionForecaster, create_forecast_summary
)
from trend_analysis import (
    calculate_growth_rate, calculate_cagr, detect_trend,
    calculate_moving_average, detect_seasonality, calculate_trend_metrics,
    identify_anomalies
)

warnings.filterwarnings('ignore')

# Set style for visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ All libraries imported successfully")

## Section 1: Load and Explore Financial Data

Loading and analyzing the raw financial data from CSV files to understand the structure and identify any data quality issues.

In [ ]:
# Load financial data
base_path = Path('../data/raw/')

financial_df = pd.read_csv(base_path / 'financial_statements.csv')
sales_df = pd.read_csv(base_path / 'sales_data.csv')
market_df = pd.read_csv(base_path / 'market_data.csv')

print("=" * 60)
print("FINANCIAL STATEMENTS DATA")
print("=" * 60)
print(f"Shape: {financial_df.shape}")
print(f"\nFirst few rows:\n{financial_df.head()}")
print(f"\nData types:\n{financial_df.dtypes}")
print(f"\nMissing values:\n{financial_df.isnull().sum()}")
print(f"\nBasic statistics:\n{financial_df.describe()}")

print("\n" + "=" * 60)
print("SALES DATA")
print("=" * 60)
print(f"Shape: {sales_df.shape}")
print(f"\nFirst few rows:\n{sales_df.head()}")
print(f"\nRevenue by Product Line:\n{sales_df.groupby('product_line')['revenue'].sum()}")

print("\n" + "=" * 60)
print("MARKET DATA")
print("=" * 60)
print(f"Shape: {market_df.shape}")
print(f"\nFirst few rows:\n{market_df.head()}")
print(f"\nDate range: {market_df['date'].min()} to {market_df['date'].max()}")

## Section 2: Calculate Financial Ratios

Computing key financial ratios to assess liquidity, profitability, and efficiency.

In [ ]:
# Calculate Financial Ratios
ratios_df = pd.DataFrame()

ratios_df['period'] = financial_df['period']
ratios_df['current_ratio'] = financial_df.apply(
    lambda row: calculate_current_ratio(row['current_assets'], row['current_liabilities']),
    axis=1
)
ratios_df['quick_ratio'] = financial_df.apply(
    lambda row: calculate_quick_ratio(row['current_assets'], row['inventory'], row['current_liabilities']),
    axis=1
)
ratios_df['profit_margin'] = financial_df.apply(
    lambda row: calculate_profit_margin(row['net_income'], row['revenue']),
    axis=1
)
ratios_df['roa'] = financial_df.apply(
    lambda row: calculate_roa(row['net_income'], row['total_assets']),
    axis=1
)
ratios_df['roe'] = financial_df.apply(
    lambda row: calculate_roe(row['net_income'], row['shareholders_equity']),
    axis=1
)
ratios_df['debt_to_equity'] = financial_df.apply(
    lambda row: calculate_debt_to_equity(row['total_debt'], row['shareholders_equity']),
    axis=1
)
ratios_df['asset_turnover'] = financial_df.apply(
    lambda row: calculate_asset_turnover(row['revenue'], row['total_assets']),
    axis=1
)

print("\n" + "=" * 100)
print("FINANCIAL RATIOS ANALYSIS")
print("=" * 100)
print(f"\n{ratios_df.to_string(index=False)}")

# Ratio Summary Statistics
print("\n" + "-" * 100)
print("RATIO SUMMARY STATISTICS")
print("-" * 100)
ratio_summary = ratios_df[['current_ratio', 'quick_ratio', 'profit_margin', 'roa', 'roe', 'debt_to_equity', 'asset_turnover']].describe()
print(ratio_summary)

## Section 3: Perform Revenue Forecasting

Using multiple time-series models to generate revenue forecasts for 2025.

In [ ]:
# Revenue Forecasting Models
revenue_data = financial_df['revenue'].astype(float)

# 1. Moving Average Model
ma_model = MovingAverageForecaster(window=3)
ma_model.fit(revenue_data)
ma_forecast = ma_model.predict(4)

# 2. Exponential Smoothing Model
es_model = ExponentialSmoothingForecaster(alpha=0.3)
es_model.fit(revenue_data)
es_forecast = es_model.predict(4)

# 3. Linear Regression Model
lr_model = LinearRegressionForecaster()
lr_model.fit(revenue_data)
lr_forecast = lr_model.predict(4)

# Create forecast summary
print("\n" + "=" * 100)
print("REVENUE FORECAST MODELS - 2025 Q1-Q4")
print("=" * 100)

# Create comparison dataframe
forecast_comparison = pd.DataFrame({
    'Quarter': ['2025-Q1', '2025-Q2', '2025-Q3', '2025-Q4'],
    'Moving Average': ma_forecast.values,
    'Exp Smoothing': es_forecast.values,
    'Linear Regression': lr_forecast.values
})

forecast_comparison['Consensus'] = forecast_comparison[['Moving Average', 'Exp Smoothing', 'Linear Regression']].mean(axis=1)

print(f"\n{forecast_comparison.to_string(index=False)}")

# Model Statistics
print("\n" + "-" * 100)
print("FORECAST SUMMARY STATISTICS")
print("-" * 100)
print(f"\nMoving Average (MA):")
print(f"  - Mean: ${ma_forecast.mean():,.0f}")
print(f"  - Range: ${ma_forecast.min():,.0f} - ${ma_forecast.max():,.0f}")

print(f"\nExponential Smoothing (ES):")
print(f"  - Mean: ${es_forecast.mean():,.0f}")
print(f"  - Range: ${es_forecast.min():,.0f} - ${es_forecast.max():,.0f}")

print(f"\nLinear Regression (LR):")
print(f"  - Mean: ${lr_forecast.mean():,.0f}")
print(f"  - Range: ${lr_forecast.min():,.0f} - ${lr_forecast.max():,.0f}")
print(f"  - Trend: ${lr_model.get_trend():,.0f} per period")

print(f"\nConsensus Forecast:")
print(f"  - Mean: ${forecast_comparison['Consensus'].mean():,.0f}")
print(f"  - Range: ${forecast_comparison['Consensus'].min():,.0f} - ${forecast_comparison['Consensus'].max():,.0f}")
print(f"  - 2025 Total: ${forecast_comparison['Consensus'].sum():,.0f}")

## Section 4: Conduct Trend Analysis

Analyzing trends, growth rates, and patterns in financial metrics.

In [ ]:
# Trend Analysis
print("\n" + "=" * 100)
print("TREND ANALYSIS")
print("=" * 100)

# Overall trend
overall_trend = detect_trend(revenue_data)
print(f"\nRevenue Trend (Overall): {overall_trend}")
print(f"Revenue Trend (Last 4 periods): {detect_trend(revenue_data.tail(4))}")

# Growth rates
print(f"\n" + "-" * 100)
print("GROWTH RATE ANALYSIS")
print("-" * 100)

# QoQ Growth
qoq_growth = []
for i in range(1, len(financial_df)):
    growth = calculate_growth_rate(financial_df.iloc[i]['revenue'], financial_df.iloc[i-1]['revenue'])
    qoq_growth.append(growth)

qoq_df = pd.DataFrame({
    'Period': financial_df.iloc[1:]['period'].values,
    'QoQ Growth %': qoq_growth
})
print(f"\nQuarter-over-Quarter Growth:\n{qoq_df.to_string(index=False)}")

# CAGR Calculation
beginning_revenue = financial_df.iloc[0]['revenue']
ending_revenue = financial_df.iloc[-1]['revenue']
periods = len(financial_df) - 1
cagr = calculate_cagr(beginning_revenue, ending_revenue, periods)
print(f"\nCompound Annual Growth Rate (2022-2024):")
print(f"  Beginning Value: ${beginning_revenue:,.0f}")
print(f"  Ending Value: ${ending_revenue:,.0f}")
print(f"  Periods: {periods} quarters ({periods/4:.1f} years)")
print(f"  CAGR: {cagr:.2f}%")

# Moving Averages
print(f"\n" + "-" * 100)
print("MOVING AVERAGES")
print("-" * 100)

sma_3 = calculate_moving_average(revenue_data, 3)
sma_4 = calculate_moving_average(revenue_data, 4)
ema_3 = calculate_exponential_moving_average(revenue_data, 3)

trend_df = pd.DataFrame({
    'Period': financial_df['period'],
    'Revenue': revenue_data.values,
    'SMA_3': sma_3.values,
    'SMA_4': sma_4.values,
    'EMA_3': ema_3.values
})
print(f"\n{trend_df.to_string(index=False)}")

# Seasonality Detection
print(f"\n" + "-" * 100)
print("SEASONALITY ANALYSIS")
print("-" * 100)
seasonality = detect_seasonality(revenue_data, period=4)
print(f"\nSeasonality Detected: {seasonality['seasonal']}")
print(f"Seasonality Strength: {seasonality['strength']:.4f}")
print(f"Seasonal Period: {seasonality['period']} quarters")

## Section 5: Visualize Financial Metrics

Creating comprehensive visualizations of financial trends and forecasts.

In [ ]:
# Create comprehensive visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Financial Analytics Dashboard', fontsize=16, fontweight='bold')

# 1. Revenue Trend with Forecasts
ax1 = axes[0, 0]
periods = range(len(financial_df))
periods_forecast = range(len(financial_df) - 1, len(financial_df) + 3)

ax1.plot(periods, revenue_data.values / 1e6, marker='o', label='Actual Revenue', linewidth=2, markersize=6)
ax1.plot(periods_forecast, list([revenue_data.iloc[-1] / 1e6]) + list(ma_forecast.values / 1e6), 
         marker='s', linestyle='--', label='MA Forecast', linewidth=2, markersize=5)
ax1.plot(periods_forecast, list([revenue_data.iloc[-1] / 1e6]) + list(es_forecast.values / 1e6), 
         marker='^', linestyle='--', label='ES Forecast', linewidth=2, markersize=5)
ax1.plot(periods_forecast, list([revenue_data.iloc[-1] / 1e6]) + list(lr_forecast.values / 1e6), 
         marker='d', linestyle='--', label='LR Forecast', linewidth=2, markersize=5)
ax1.set_xlabel('Period (Quarters)', fontweight='bold')
ax1.set_ylabel('Revenue ($ Millions)', fontweight='bold')
ax1.set_title('Revenue Trend & Forecasts', fontweight='bold')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.set_xticks(range(0, len(financial_df) + 4, 2))

# 2. Financial Ratios Heatmap
ax2 = axes[0, 1]
ratio_data = ratios_df[['current_ratio', 'quick_ratio', 'profit_margin', 'roa', 'roe', 'debt_to_equity', 'asset_turnover']].values
sns.heatmap(ratio_data.T, annot=True, fmt='.2f', cmap='RdYlGn', ax=ax2, cbar_kws={'label': 'Ratio Value'})
ax2.set_yticklabels(['Current Ratio', 'Quick Ratio', 'Profit Margin', 'ROA', 'ROE', 'D/E Ratio', 'Asset Turnover'], rotation=0)
ax2.set_xlabel('Period', fontweight='bold')
ax2.set_title('Financial Ratios Heatmap', fontweight='bold')

# 3. Profitability Metrics Trend
ax3 = axes[1, 0]
ax3.plot(range(len(ratios_df)), ratios_df['profit_margin'], marker='o', label='Profit Margin %', linewidth=2)
ax3.plot(range(len(ratios_df)), ratios_df['roa'], marker='s', label='ROA %', linewidth=2)
ax3.plot(range(len(ratios_df)), ratios_df['roe'], marker='^', label='ROE %', linewidth=2)
ax3.set_xlabel('Period', fontweight='bold')
ax3.set_ylabel('Percentage (%)', fontweight='bold')
ax3.set_title('Profitability Metrics Trend', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Liquidity vs Leverage
ax4 = axes[1, 1]
x_pos = np.arange(len(ratios_df))
width = 0.35
ax4.bar(x_pos - width/2, ratios_df['current_ratio'], width, label='Current Ratio', alpha=0.8)
ax4_twin = ax4.twinx()
ax4_twin.plot(x_pos, ratios_df['debt_to_equity'], marker='o', color='red', label='D/E Ratio', linewidth=2, markersize=6)
ax4.set_xlabel('Period', fontweight='bold')
ax4.set_ylabel('Current Ratio', fontweight='bold')
ax4_twin.set_ylabel('Debt-to-Equity Ratio', fontweight='bold', color='red')
ax4.set_title('Liquidity vs Leverage Analysis', fontweight='bold')
ax4.legend(loc='upper left')
ax4_twin.legend(loc='upper right')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Visualizations generated successfully")

## Section 6: Generate Financial Insights Report

Summarizing key findings, insights, and strategic recommendations.

In [ ]:
# Generate Comprehensive Insights Report
print("\n" + "=" * 100)
print("EXECUTIVE SUMMARY - KEY INSIGHTS & RECOMMENDATIONS")
print("=" * 100)

# Key Metrics
print(f"\n1. REVENUE PERFORMANCE")
print(f"   • Current Revenue (Latest): ${ending_revenue:,.0f}")
print(f"   • Historical CAGR: {cagr:.2f}%")
print(f"   • 2025 Consensus Forecast: ${forecast_comparison['Consensus'].sum():,.0f}")
print(f"   • Expected Growth Rate: {((forecast_comparison['Consensus'].sum() / (ending_revenue * 4)) - 1) * 100:.2f}%")

# Profitability
print(f"\n2. PROFITABILITY ANALYSIS")
latest_profit_margin = ratios_df.iloc[-1]['profit_margin']
latest_roe = ratios_df.iloc[-1]['roe']
latest_roa = ratios_df.iloc[-1]['roa']
print(f"   • Current Profit Margin: {latest_profit_margin:.2f}%")
print(f"   • Current ROE: {latest_roe:.2f}%")
print(f"   • Current ROA: {latest_roa:.2f}%")
print(f"   • Trend: {'↑ Improving' if latest_roa > ratios_df.iloc[0]['roa'] else '↓ Declining'}")

# Liquidity
print(f"\n3. LIQUIDITY POSITION")
latest_current = ratios_df.iloc[-1]['current_ratio']
latest_quick = ratios_df.iloc[-1]['quick_ratio']
print(f"   • Current Ratio: {latest_current:.2f} (Healthy threshold: 1.5-3.0)")
print(f"   • Quick Ratio: {latest_quick:.2f} (Healthy threshold: 0.5-1.0)")
print(f"   • Status: {'✓ Strong' if latest_current > 1.5 else '⚠ Adequate'}")

# Leverage
print(f"\n4. CAPITAL STRUCTURE")
latest_de = ratios_df.iloc[-1]['debt_to_equity']
print(f"   • Debt-to-Equity Ratio: {latest_de:.2f}")
print(f"   • Risk Level: {'Conservative' if latest_de < 1.0 else 'Moderate'}")
print(f"   • Borrowing Capacity: Available")

# Efficiency
print(f"\n5. OPERATIONAL EFFICIENCY")
latest_at = ratios_df.iloc[-1]['asset_turnover']
at_trend = latest_at - ratios_df.iloc[0]['asset_turnover']
print(f"   • Asset Turnover: {latest_at:.3f}")
print(f"   • Trend Change: {at_trend:+.3f} {'(Improving)' if at_trend > 0 else '(Declining)'}")
print(f"   • Status: {'✓ Improving' if at_trend > 0 else '⚠ Monitoring needed'}")

# Strategic Recommendations
print(f"\n" + "-" * 100)
print("STRATEGIC RECOMMENDATIONS")
print("-" * 100)
recommendations = [
    "1. GROWTH: Continue revenue growth strategy - strong uptrend demonstrates market acceptance",
    "2. PROFITABILITY: Monitor margins closely; maintain cost discipline amid potential inflation",
    "3. LIQUIDITY: Current position is healthy; focus on efficient working capital management",
    "4. INVESTMENT: Conservative debt levels allow room for strategic capital investments",
    "5. EFFICIENCY: Asset turnover improving - opportunity for targeted operational optimization",
    "6. FORECASTING: Use consensus forecast for planning; monitor quarterly actuals vs forecast",
    "7. MARKET: Track external factors (interest rates, inflation, consumer confidence)"
]

for rec in recommendations:
    print(f"   • {rec}")

print(f"\n" + "=" * 100)